# 2/5/2024 랩미팅
## goal
* 워크샵 버전 (DP 모델링을 파이썬에서 함) -> 업데이트: DP 모델은 maude로 하고, python maude binding을 이용하여 강화학습
* progress: 재현 성공

## key idea
#### Propositional Interface
- 현재까지 관찰 결과, DP(5)의 학습결과를 DP(6)에 "이어붙일" 수 있었음. (성능과 별개로. 이 경우는 성능"까지" 성공적으로 전달했음)
- 왜냐면 두 모델이 AP = { E.Think, E.Hungry, E.Single, E.Eat } 를 "공유"하기 때문
- 즉, 위의 AP는 "인터페이스"로서 개념적으로 M = { DP(5), DP(6), ... } 라는 "(possibly infinite) class of models"를 denote하는 역할

- 과연 parameterized model이 아닌 전혀 다른 모델끼리도 이 접근이 유효할까? e.g. Paxos -> Raft
- 예를 들어, M = { consensus protocols }를 포착하는 AP를 찾을 수 있어서 그것을 인터페이스로 하는 휴리스틱 러닝이 가능할까?

=> 인터페이스를 공유하는 maude model 2개만 얻으면 아래 코드를 거의 그대로 이용해서 실험해볼 수 있을듯

#### Abstraction & Refinement
- configuration은 AP를 feature로 하는 feature vector로 인코딩됨.
- 당연히 명제 p가 추가되면 refinement고 삭제되면 abstraction임.
- 예를 들어 {p,q,r}을 사용한다면 특정 상태는 3차원 불리언벡터이므로 8 x 8 테이블이 학습됨
- 학습된 테이블을 잘 관찰해보면 q들 사이에 모종의 패턴이 있을 수가 있음 (아래 코드의 출력 참고)

- 직관적으로, 아귀만 맞으면 비슷한 q값을 가지는 셀을(즉, index를) merge할 수 있지 않을까? e.g. 000 ~ 001 로 merge
- 000, 001, ..., 111 총 8개의 엔트리가 있을때 "가장 좋은" pairwise matching을 찾는 문제
- index의 best matching을 찾으면 그 결과를 예를 들어 {p, q, r} -> {p or q, q and r} 등으로 symbolic 하게 대응 시킬 수 있음.
- 토너먼트 형식으로 1라운드씩 할때마다 명제의 개수는 하나씩 줄어들음 (abstraction step)
- 그렇게 해서 얻어진 p or q, q and r은 일종의 "학습된 패턴"의 역할을 하지 않을까?

```
fmod DINING-PHILOSOPHERS-FUNCS is
  protecting NAT .
  sorts Status .    
  ops think hungry single eat : -> Status [ctor] .

  op #N : ~> Nat .  --- total number of philosophers
  vars I J : Nat .  
  
  ops lc rc : Nat ~> Nat .    
  eq lc(I) = I .    
  ceq rc(I) = s(I) if (s(I) < #N) = true .
  ceq rc(I) = 0 if (s(I) == #N) = true .

  op adj : Nat Nat ~> Bool .
  eq adj(I,J) = (J == lc(I)) or (J == rc(I)) .
endfm

mod DINING-PHILOSOPHERS is
  including DINING-PHILOSOPHERS-FUNCS .

  sorts Phil Chop Conf .
  subsorts Phil Chop < Conf .   

  op p : Nat Status -> Phil [ctor] .
  op c : Nat -> Chop [ctor] .    
  
  op none : -> Conf [ctor] .
  op _||_ : Conf Conf -> Conf [ctor comm assoc id: none] .

  --- eq #N = s(s(s(0))) . 
  
  vars I J : Nat .	var C : Conf .	
  
  *** defining the system behavior with localized fairness
  *** NOTE: rule attributes also declare the simplified version of
  *** spatial action patterns, e.g., wake(I) = {'wake : 'I \ I}

  rl [th]: p(I,think) => p(I,hungry) .

 crl [hs]: p(I,hungry) || c(J) => p(I,single) 
   if adj(I,J) = true .

 crl [se]: p(I,single) || c(J) => p(I,eat) 
   if adj(I,J) = true .

  rl [et]: p(I,eat) => p(I,think) || c(lc(I)) || c(rc(I)) .
endm

mod ABSTRACT-DINING-PHILOSOPHERS is
  including DINING-PHILOSOPHERS .

  sort AIdx .

  ops hasThink hasHungry hasSingle hasEat : Conf -> Bool .
  op aidx : Conf -> Nat .

  var I : Nat . var C : Conf . var S : Status .

  eq hasThink(p(I,think) || C) = true .
  eq hasThink(p(I,S) || C) = hasThink(C) [owise] .
  eq hasThink(c(I) || C) = hasThink(C) .
  eq hasThink(none) = false .

  eq hasHungry(p(I,hungry) || C) = true .
  eq hasHungry(p(I,S) || C) = hasHungry(C) [owise] .
  eq hasHungry(c(I) || C) = hasHungry(C) .
  eq hasHungry(none) = false .

  eq hasSingle(p(I,single) || C) = true .
  eq hasSingle(p(I,S) || C) = hasSingle(C) [owise] .
  eq hasSingle(c(I) || C) = hasSingle(C) .
  eq hasSingle(none) = false .

  eq hasEat(p(I,eat) || C) = true .
  eq hasEat(p(I,S) || C) = hasEat(C) [owise] .
  eq hasEat(c(I) || C) = hasEat(C) .
  eq hasEat(none) = false .
 
  eq aidx(C) = 8 * if hasThink(C) then 1 else 0 fi + 4 * if hasHungry(C) then 1 else 0 fi + 2 * if hasSingle(C) then 1 else 0 fi + 1 * if hasEat(C) then 1 else 0 fi .
endm

mod DP5 is
  including ABSTRACT-DINING-PHILOSOPHERS .
  
  eq #N = 5 .
endm
```

In [1]:
import maude
import random
import numpy as np
import numpy.ma as ma
import tqdm
from tqdm.notebook import tqdm

In [2]:
maude.init()
maude.load('./dp.maude')
m = maude.getCurrentModule()
print('Using', m, 'module')

Using DP5 module


In [3]:
def dp_generator():
    N = random.choice([5])
    P = [0] * N #np.zeros(N, dtype=int)
    C = [1] * N #np.ones(N, dtype=int)

    for i in range(N):
        c = random.randrange(3)
        if c == 1:
            # to left
            P[(i-1) % N] += 1
            C[i] = 0
        elif c == 2:
            # to right:
            P[i] += 1
            C[i] = 0

    # here, self.P[i] denotes the number of chopstics assigned for ith philos
    for i in range(N):
        if P[i] == 0:
            P[i] = random.randrange(2) # either think or hungry
        else:
            P[i] += 1 # one chopstick or eat
    
    s = []
    for i in range(N):
        if P[i] == 0:
            s.append(f'p({i},think)')
        elif P[i] == 1:
            s.append(f'p({i},hungry)')
        elif P[i] == 2:
            s.append(f'p({i},single)')
        elif P[i] == 3:
            s.append(f'p({i},eat)')
            
        if C[i] == 0:
            pass
        elif C[i] == 1:
            s.append(f'c({i})')
    return m.parseTerm(' || '.join(s))

dp_generator()

p(0, single) || p(1, think) || c(1) || p(2, single) || c(2) || p(3, think) || p(4, hungry) || c(4)

MaudeEnv does not know anything about the model in consideration

In [4]:
class MaudeEnv():
    def __init__(self, g, init_term=None):
        self.conf_gen = g
        self.n_pred = 4
        self.reset(init_term)
        
        #self.term = t
        #self.aidx = self.get_aidx(t)
        #self.nbrs = [(t,self.get_aidx(t)) for t,_,_,_ in t.search(1, m.parseTerm('X:Conf'), depth = 1)]
        
    def reset(self, init_term=None):
        if init_term == None:
            t = self.conf_gen()
        else:
            t = init_term
        self.term = t
        self.aidx = self.get_aidx(t)
        #self.nbrs = [t for t,_,_,_ in t.search(1, m.parseTerm('X:Conf'), depth = 1)]
        self.nbrs = [(t,self.get_aidx(t)) for t,_,_,_ in t.search(1, m.parseTerm('X:Conf'), depth = 1)]
        
        self.available_actions = [action for (_,action) in self.nbrs]
        self.mask = np.ones(2 ** self.n_pred, dtype=int)
        self.mask[self.available_actions] = 0
        return {
            'term' : self.term,
            'aidx' : self.aidx,
            'nbrs' : self.nbrs,
            'mask' : self.mask
        }
        
    def step(self, action):
        pairs = [(term, aidx) for (term,aidx) in self.nbrs if aidx == action]
        if pairs == []:
            raise Exception("invalid action")
        #nbrs = [t for t in self.nbrs if self.get_aidx(t) == action]
        #if nbrs == []:
        #    raise Exception("invalid action")
        state = self.reset(random.choice(pairs)[0])
        done = True if self.available_actions == [] else False
        reward = 1 if done else 0
        return state, reward, done
    
    def get_aidx(self, t):
        aidx = m.parseTerm('aidx(' + t.prettyPrint(0) + ')')
        aidx.reduce()
        return aidx.toInt()

In [23]:
def abs_greedy_policy(Qtable, state):
    # Exploitation: take the action with the highest state, action value
    aidx = state["aidx"]
    mask = state["mask"]
    #mask = np.where(True, mask^1, mask) # flip 0 & 1
    if 0 in mask:
        masked_Q = ma.masked_array(Qtable[aidx][:], mask=mask)
        action = np.argmax(masked_Q)
    else:
        action = -1 # deadlock
    return action

def abs_epsilon_greedy_policy(Qtable, state, epsilon):
    # Randomly generate a number between 0 and 1
    random_num = random.uniform(0, 1)
    # if random_num > greater than epsilon --> exploitation
    if random_num > epsilon:
        # Take the action with the highest value given a state
        # np.argmax can be useful here
        action = abs_greedy_policy(Qtable, state)
    # else --> exploration
    else:
        nbrs = state["nbrs"]
        if nbrs != []:
            action = random.choice(nbrs)[1]
        else:
            action = -1
    return action

def abs_train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        # Reset the environment
        state = env.reset()
        step = 0
        done = False

        # repeat
        for step in range(max_steps):
            # Choose the action At using epsilon greedy policy
            s = state["aidx"]
            a = abs_epsilon_greedy_policy(Qtable, state, epsilon)
            
            # assert action not -1
            if a == -1:
                break

            # Take action At and observe Rt+1 and St+1
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done = env.step(a)

            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]

            Qtable[s][a] = Qtable[s][a] + learning_rate * (
                reward + gamma * np.max(Qtable[a]) - Qtable[s][a]
            )

            # If terminated or truncated finish the episode
            if done:
                break

            # Our next state is the new state
            state = new_state
    print('training done!')
    return Qtable

In [24]:
# Training parameters
n_training_episodes = 1000  # Total training episodes
learning_rate = 0.7  # Learning rate

# Evaluation parameters
n_eval_episodes = 100  # Total number of test episodes

# Environment parameters
#env_id = "FrozenLake-v1"  # Name of the environment
max_steps = 300  # Max steps per episode
gamma = 0.95  # Discounting rate
eval_seed = []  # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.05  # Minimum exploration probability
decay_rate = 0.0005  # Exponential decay rate for exploration prob

In [25]:
env = MaudeEnv(dp_generator)
Qtable = np.zeros(shape=(16,16))
Qtable = abs_train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable)

  0%|          | 0/1000 [00:00<?, ?it/s]

training done!


In [26]:
print('absQtable density:', np.count_nonzero(Qtable), '/', Qtable.size)
print(Qtable.shape)
print(Qtable)

absQtable density: 43 / 256
(16, 16)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.665      0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.857375   0.         0.         0.         0.
  0.8573

In [27]:
# e-greedy Simulator
def abs_simulate(Qtable, max_steps=1000, epsilon=0.01):
    state = env.reset()

    for step in range(max_steps):
        # Choose the action At using epsilon greedy policy
        action = abs_epsilon_greedy_policy(Qtable, state, epsilon)
        if action == -1:
            break

        # Take action At and observe Rt+1 and St+1
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done = env.step(action)

        # If terminated or truncated finish the episode
        if done:
            break

        # Our next state is the new state
        state = new_state
        
    return step + 1

In [29]:
max_steps = 1000
res_trained = []
res_random = []
for _ in tqdm(range(100)):
    num_steps_trained = abs_simulate(Qtable, max_steps=max_steps, epsilon=0)
    num_steps_random = abs_simulate(Qtable, max_steps=max_steps, epsilon=1)
    res_trained.append(num_steps_trained)
    res_random.append(num_steps_random)
print('trained:')
print(res_trained)
print('avg:')
print(np.average(res_trained))
print('=====')
print('random:')
print(res_random)
print('avg:')
print(np.average(res_random))

  0%|          | 0/100 [00:00<?, ?it/s]

trained:
[17, 13, 1, 2, 23, 16, 34, 11, 38, 33, 8, 43, 12, 37, 10, 22, 3, 22, 64, 16, 10, 7, 67, 26, 15, 12, 4, 16, 13, 30, 39, 6, 6, 10, 9, 11, 17, 22, 13, 15, 47, 24, 24, 13, 17, 62, 54, 20, 10, 12, 6, 28, 8, 19, 38, 67, 8, 9, 24, 15, 28, 5, 21, 13, 17, 5, 4, 3, 35, 14, 8, 9, 48, 6, 22, 1, 13, 17, 5, 6, 7, 13, 27, 12, 4, 7, 7, 6, 30, 23, 4, 84, 20, 1, 5, 7, 13, 58, 20, 7]
avg:
19.13
=====
random:
[30, 62, 133, 115, 26, 153, 19, 3, 86, 125, 65, 102, 213, 88, 122, 25, 37, 17, 62, 197, 20, 89, 77, 5, 228, 127, 22, 12, 91, 1, 198, 186, 105, 104, 100, 137, 81, 60, 64, 8, 138, 179, 37, 57, 115, 45, 116, 12, 46, 11, 105, 17, 12, 113, 130, 23, 153, 63, 87, 109, 56, 59, 139, 21, 77, 78, 122, 9, 80, 58, 120, 279, 31, 10, 157, 75, 31, 146, 160, 23, 95, 175, 96, 139, 2, 175, 144, 20, 108, 38, 37, 5, 130, 21, 63, 3, 12, 84, 52, 48]
avg:
81.41


In [152]:
env = MaudeEnv(dp_generator)
env.mask

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1])

In [8]:
Qtable = np.zeros(shape=(16,16))
print(Qtable)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [95]:
#t = m.parseTerm("p(0,think) || c(0) || p(s(0),think) || c(s(0)) || p(s(s(0)),think) || c(s(s(0)))")
env = MaudeEnv(dp_generator)

while env.available_actions != []:
    print('term:', env.term)
    #print('aidx:', env.aidx)
    action = random.choice(env.available_actions)
    print('action (next aidx):', action)
    state, reward, done = env.step(action)
    print(f'action: {action}, state: {state}, reward: {reward}, done: {done}')
    print('===')
print('last term:', env.term)

term: c(1) || c(3) || c(4) || c(5) || p(0, single) || p(1, think) || p(2, single) || p(3, think) || p(4, think) || p(5, hungry)
action (next aidx): 14
action: 14, state: {'term': c(1) || c(3) || c(4) || c(5) || p(0, single) || p(1, hungry) || p(2, single) || p(3, think) || p(4, think) || p(5, hungry), 'aidx': 14, 'nbrs': [(c(3) || c(4) || c(5) || p(0, single) || p(1, single) || p(2, single) || p(3, think) || p(4, think) || p(5, hungry), 14), (c(1) || c(3) || c(4) || p(0, single) || p(1, hungry) || p(2, single) || p(3, think) || p(4, think) || p(5, single), 14), (c(3) || c(4) || c(5) || p(0, eat) || p(1, hungry) || p(2, single) || p(3, think) || p(4, think) || p(5, hungry), 15), (c(1) || c(3) || c(4) || c(5) || p(0, single) || p(1, hungry) || p(2, single) || p(3, hungry) || p(4, think) || p(5, hungry), 14), (c(1) || c(3) || c(4) || c(5) || p(0, single) || p(1, hungry) || p(2, single) || p(3, think) || p(4, hungry) || p(5, hungry), 14)], 'mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [10]:
tterm = m.parseTerm('p(0, single) || p(1, hungry) || p(2, eat) || p(3, hungry) || p(4, eat)')
print(tterm)
tenv = MaudeEnv(dp_generator, tterm)
state = tenv.reset(tterm)
#print(state)
a = abs_greedy_policy(Qtable, state)
print(a)
state, reward, done = tenv.step(a)
print(state, done)
#print(abs_epsilon_greedy_policy(Qtable, state, 0.5))

p(0, single) || p(1, hungry) || p(2, eat) || p(3, hungry) || p(4, eat)
15
{'term': c(2) || c(3) || p(0, single) || p(1, hungry) || p(2, think) || p(3, hungry) || p(4, eat), 'aidx': 15, 'nbrs': [(c(3) || p(0, single) || p(1, single) || p(2, think) || p(3, hungry) || p(4, eat), 15), (c(2) || p(0, single) || p(1, hungry) || p(2, think) || p(3, single) || p(4, eat), 15), (c(2) || c(3) || p(0, single) || p(1, hungry) || p(2, hungry) || p(3, hungry) || p(4, eat), 7), (c(0) || c(2) || c(3) || c(4) || p(0, single) || p(1, hungry) || p(2, think) || p(3, hungry) || p(4, think), 14)], 'mask': array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0])} False
